## Import Statements

In [2]:
import gensim.downloader as api
import numpy as np
import re
import csv
import pandas as pd
import pprint
import string
import nltk
import sys

from IPython.display import HTML
from nltk.corpus import wordnet 
from sklearn.manifold import TSNE
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize

pathToDatasets = '../datasets/'
pathToDataScripts = '../datasets/scripts/'
filePath = '../datasets/GoogleNews-vectors-negative300.bin'

sys.path.insert(0, pathToDataScripts)
from cleanDataset import tokenize_words 

## Downloading binaries and models


In [3]:
word_vectors = api.load("glove-wiki-gigaword-100")
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/ubuntu/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ubuntu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Global Variables and Global Objects

In [76]:
senty = SentimentIntensityAnalyzer()
vocabulary = word_vectors.vocab;


NUMBER_OF_ALTERNATIVES = 7
TWEET_START = 50
NUM_OF_TWEETS = 25

punctuation = r"\"#$%&'()+-/:;<=>?@[\]*^_`{|}~"

## Class for Sentences


In [36]:
class Sentence:
    
    def __init__(self, sentence, sentiment):
        self.ogSentence = sentence;
        self.ogSentiment = sentiment;
        self.indexToSetOfWords = {}
        self.alternateSentences = [];
        self.alternateHTMLs = [];
        self.finalShiftSentences = [];

    def addAlternativesByIndex(self, index, listOfAlternatives):
        '''
            Adds the list of possible alternative words that 
            can be used per word based on the index of the word in the tokenized 
            sentence. (from cleanAndTokenizeText())
        '''
        if(self.indexToSetOfWords.get(index)):
            self.indexToSetOfWords[index] = self.indexToSetOfWords.union(set(listOfAlternatives))
        else:
            self.indexToSetOfWords[index] = set(listOfAlternatives)
        
    def addAlternativeStrings(self, strings):
        if(isinstance(strings,str)):
            self.alternateSentences.append(strings)
        else:
            self.alternateSentences.extend(strings)
    
    def addHTMLStrings(self, htmls):
        if(isinstance(htmls, str)):
            self.alternateHTMLS.append(htmls)
        else:
            self.alternateHTMLs.extend(htmls)
    
    def addFinalSentences(self, sentences):
        self.finalShiftSentences.extend(sentences)
        if(isinstance(sentences, str)):
            self.finalShiftSentences.append(sentences)
        else:
            self.finalShiftSentences.extend(sentences)
            

## Utility Code

In [80]:
def cstr(s, color='black'):
    return "<text style=color:{}>{}</text>".format(color, s)

def cleanAndTokenizeText(text):
    text = text.lower();
    newString = ""
    for char in text:
        if char not in punctuation:
            newString += char
    text = word_tokenize(newString)
    return text;

def getPOSTags(tweet):
    tags = nltk.pos_tag(tweet)
    return tags;

def getAntonymsOfWords(word):
    if(word not in vocabulary):
        return []
    setOfAntonyms = set()
    for syn in wordnet.synsets(word):
        for l in syn.lemmas():
            anton = l.antonyms()
            if(anton!=[]):
                setOfAntonyms.add(anton[0].name())
    return list(setOfAntonyms)

def listReplacements(word):
    if(word not in vocabulary):
        print(" --- {0} not in vocabulary ---".format(word))
        return []
    possibleReplacements = [word[0] for word in word_vectors.most_similar(word,topn=NUMBER_OF_ALTERNATIVES)]
    if(possibleReplacements == []):
        print(" --- No replacements for word {0} ---".format(word))
    antonyms = getAntonymsOfWords(word)
    if(antonyms != []):
        possibleReplacements.extend(antonyms)
        return possibleReplacements
    return []
    
def posApprovedReplacements(alternativeWords, userTokens, indexOfToken):
    if(alternativeWords == []):
        return []
    tempTokens = userTokens[:]
    POSTokens = getPOSTags(tempTokens)
    validWords = []
    
    mainTag = POSTokens[indexOfToken][1]
    mainWord = userTokens[indexOfToken]
    
    for ind,word in enumerate(alternativeWords):
        tempTokens[indexOfToken] = word
        posTags = getPOSTags(tempTokens)
        newTag = (posTags[indexOfToken])[1]
        
        if(str(newTag) == str(mainTag)):
            print("Word {0}[{1}] replaced with {2}[{3}]".format(mainWord, mainTag, word,newTag))
            validWords.append(word)
    return validWords
        
    
    
def getAlternativeSentences(sentenceObj):
    mainSentence = sentenceObj.ogSentence;
    mainSentiment = sentenceObj.ogSentiment;
    
    sentenceTokens = cleanAndTokenizeText(mainSentence)
   
    for ind, word in enumerate(sentenceTokens):
        alternativeStrings = []
        alternativeHTMLs = []
        
        score = senty.polarity_scores(word)['compound']
        copyOfTokens = sentenceTokens[:]
        replacements = []
        if(score != 0.0):
            tempReplacements = listReplacements(word) # get embedding based relations
            if(tempReplacements == []):
                continue
            replacements = posApprovedReplacements(tempReplacements[:], copyOfTokens[:], ind)
            if(replacements == []):
                print(" -- No POS approved words! -- for word {0}\n some non-POS:{1}".format(word, tempReplacements[:4]))
                continue
            sentenceObj.addAlternativesByIndex(ind, replacements)
            
            ## Generate new sentences by switching that word
            for newWord in replacements:
                htmlFriendlyTokens = copyOfTokens[:]
                copyOfTokens[ind] = newWord
                htmlFriendlyTokens[ind] = cstr(" <i>[{0}]</i>".format(newWord), "blue");
                newString = ' '.join(copyOfTokens)
                alternativeStrings.append(newString);
                alternativeHTMLs.append(' '.join(htmlFriendlyTokens))
        sentenceObj.addAlternativeStrings(alternativeStrings)
        sentenceObj.addHTMLStrings(alternativeHTMLs)
    return sentenceObj



## Main Cell


In [81]:
def shiftSentiment(sentenceObj, positive=True):
    
    actualTweet = sentenceObj.ogSentence;
    mainSentiment = sentenceObj.ogSentiment;
    alternateTweets = sentenceObj.alternateSentences;
    
    
    happiestTweet = ""
    saddestTweet = ""
    happiestScore = -sys.maxsize - 1
    saddestScore = sys.maxsize
    correctTweets = []
    
    for tweet in alternateTweets:
        newSenty = senty.polarity_scores(tweet)['compound']
        
        if(newSenty < saddestScore):
            saddestTweet = tweet
            saddestScore = newSenty
        if(newSenty > happiestScore):
            happiestTweet = tweet
            happiestScore = newSenty
            
        if(positive):
            if(newSenty > mainSentiment):
                correctTweets.append(tweet)
                continue
            
            elif(newSenty < mainSentiment):
                continue
                # Grab happiest tweet and if it's not "", then generate more happy tweets from it
        if(positive == False):
            if(newSenty < mainSentiment):
                correctTweets.append(tweet)
                continue
            
            elif(newSenty > mainSentiment):
                continue
                # grab happiest tweet, and if it's not "", then generate more happy tweets from it
                
#     if(correctTweets == []):
#         print("\n\nNo tweets found when trying to do Positive={}\n\n".format(positive))
    
    sentenceObj.addFinalSentences(correctTweets);
    return sentenceObj;
        
def printStrings(sentenceObj):
    newStringsHTML = sentenceObj.alternateHTMLs;
    newStrings = sentenceObj.alternateSentences;
    mainSentiment = sentenceObj.ogSentiment;    
    for ind,alteredTweet in enumerate(newStrings):
            sentimentOfNewString = senty.polarity_scores(alteredTweet)['compound']
            htmlText = newStringsHTML[ind]
            if(sentimentOfNewString == mainSentiment or sentimentOfNewString == 0):
                display(HTML(cstr("{0}: {1}".format(htmlText,sentimentOfNewString),'DarkGray')))
            elif(sentimentOfNewString > mainSentiment):
                display(HTML(cstr("{0}: {1}".format(htmlText,sentimentOfNewString),'green')))
            else:
                display(HTML(cstr("{0}: {1}".format(htmlText,sentimentOfNewString),'red')))

In [82]:
def runThroughTweets():
    
    tweets_df = pd.read_csv( pathToDatasets + 'cleanedTweets.csv' , nrows=NUM_OF_TWEETS, skiprows=TWEET_START)

    tweets = tweets_df.values

    listOfObjects = []
    for counter,tweet in enumerate(tweets):
        tweet = tweet[0]
        mainSentiment = senty.polarity_scores(tweet)['compound']
        if(mainSentiment == 0):
            continue
        print("\n {0}:{1}\n".format(tweet,mainSentiment))   
        sentenceObj = Sentence(tweet, mainSentiment)
        sentenceObj = getAlternativeSentences(sentenceObj)
        alternateTweets = (sentenceObj.alternateSentences)[:]
        if(alternateTweets == [] or alternateTweets == None):
            print(" -- No new Strings generated ---\n\n")
            continue
#         sentenceObj = shiftSentiment(sentenceObj, True);
#         printStrings(sentenceObj)
        sentenceObj = shiftSentiment(sentenceObj, False);
        printStrings(sentenceObj)
    
runThroughTweets()
        
        
    


 broadband plan a massive broken promise  via  still waiting for broadband we are :-0.2023

Word broken[NN] replaced with breaking[NN]
Word broken[NN] replaced with broke[NN]
Word broken[NN] replaced with apart[NN]
Word broken[NN] replaced with neck[NN]
Word broken[NN] replaced with conform_to[NN]
Word broken[NN] replaced with make[NN]
Word broken[NN] replaced with keep[NN]
Word broken[NN] replaced with promote[NN]
Word broken[NN] replaced with repair[NN]



  wow tons of replies from you may have to unfollow so i can see my friends tweets you are scrolling the feed a lot. :0.7845

Word friends[NNS] replaced with parents[NNS]
Word friends[NNS] replaced with relatives[NNS]
Word friends[NNS] replaced with others[NNS]



 put vacation photos online a few yrs ago. pc crashed and now i forget the name of the site. :-0.2263

Word forget[VBP] replaced with remember[VBP]
Word forget[VBP] replaced with tell[VBP]
Word forget[VBP] replaced with imagine[VBP]
Word forget[VBP] replaced with know[VBP]
Word forget[VBP] replaced with remember[VBP]
Word forget[VBP] replaced with mind[VBP]



 i need a hug :0.4767

 -- No new Strings generated ---



  not sure what they are only that they are pos! as much as i want to i dont think can trade away company assets sorry andy! :-0.2134

Word sure[JJ] replaced with unsure[JJ]
Word sure[JJ] replaced with uncertain[JJ]
Word assets[NNS] replaced with investments[NNS]
Word assets[NNS] replaced with funds[NNS]
Word assets[NNS] replaced with debts[NNS]
Word sorry[VBP] replaced with 'm[VBP]
Word sorry[VBP] replaced with glad[VBP]
Word sorry[VBP] replaced with regret[VBP]



  i hate when that happens... :-0.5719

Word hate[NN] replaced with anyone[NN]
Word hate[NN] replaced with crime[NN]


...


 mo jobs no money.  how in the hell is min wage here 4 fn clams an hour:-0.7783

Word no[DT] replaced with any[DT]
Word no[DT] replaced with all[DT]
Word hell[NN] replaced with heaven[NN]
Word hell[NN] replaced with crazy[NN]



  agreed i saw the failwhale allllll day today. :0.2732

 -- No POS approved words! -- for word agreed
 some non-POS:['agree', 'agreement', 'decided', 'agreeing']
 -- No new Strings generated ---



  oh! haha... dude i dont really look at em unless someone says hey i added you. sorry  i am so terrible at that. i need a pop up!:-0.6459

Word sorry[VB] replaced with glad[VB]
Word sorry[VB] replaced with sad[VB]
Word sorry[VB] replaced with regret[VB]



  i am sure you are right...    i need to start working out with you and the nikster... or jared at least!:0.3802

Word sure[JJ] replaced with uncertain[JJ]



 i really hate how people diss my bands!  trace is clearly not ugly!:0.2484

Word hate[VB] replaced with racist[VB]
Word hate[VB] replaced with racism[VB]
Word hate[VB] replaced with fear[VB]
Word hate[VB] replaced with bigotry[VB]
Word hate[VB] replaced with love[VB]
Word clearly[RB] replaced with obviously[RB]
Word clearly[RB] replaced with certainly[RB]
Word clearly[RB] replaced with indeed[RB]
Word clearly[RB] replaced with nonetheless[RB]
Word clearly[RB] replaced with unintelligibly[RB]
 -- No POS approved words! -- for word ugly
 some non-POS:['nasty', 'awful', 'horrible', 'stupid']



 gym attire today was puma singlet adidas shorts.......and black business socks and leather shoes  lucky did not run into any cute girls.:0.7003

Word lucky[RB] replaced with unlucky[RB]
Word lucky[RB] replaced with maybe[RB]
Word lucky[RB] replaced with unlucky[RB]
